In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/mnt/c/Users/shrey/Credit_Card_Applications.csv')

In [3]:
df.head()

,CustomerID,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,Class
0,15776156,1,22.08,11.46,2,4,4,1.585,0,0,0,1,2,100,1213,0
1,15739548,0,22.67,7.00,2,8,4,0.165,0,0,0,0,2,160,1,0
2,15662854,0,29.58,1.75,1,4,4,1.250,0,0,0,1,2,280,1,0
3,15687688,0,21.67,11.50,1,5,3,0.000,1,1,11,1,2,0,1,1
4,15715750,1,20.17,8.17,2,6,4,1.960,1,1,14,0,2,60,159,1


In [5]:
import xgboost as xgb

In [6]:
df1 = df.drop(['CustomerID','Class'],axis = 1)

In [9]:
from hyperopt import Trials, STATUS_OK, tpe, hp, fmin

In [34]:
def objective(space):

    #warnings.filterwarnings(action='ignore', category=DeprecationWarning)
    classifier = xgb.XGBClassifier(base_score = 0.5 , n_estimators = space['n_estimators'],
                            max_depth = int(space['max_depth']),
                            learning_rate = space['learning_rate'],
                            #gamma = space['gamma'],
                            min_child_weight = space['min_child_weight'],
                            subsample = space['subsample'],
                            colsample_bytree = space['colsample_bytree']
                            )
    
    classifier.fit(df1, df['Class'])

    # Applying k-Fold Cross Validation
    from sklearn.model_selection import cross_val_score
    accuracies = cross_val_score(estimator = classifier, X = df1, y = df['Class'], cv = 10)
    CrossValMean = accuracies.mean()

    #print("CrossValMean:", CrossValMean)

    return{'loss':1-CrossValMean, 'status': STATUS_OK }

space = {
    'max_depth' : hp.choice('max_depth', range(3, 10, 1)),
    'learning_rate' : hp.quniform('learning_rate', 0.01, 0.5, 0.01),
    'n_estimators' : hp.choice('n_estimators', range(100, 205, 5)),
    'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 1),
    'subsample' : hp.quniform('subsample', 0.1, 1, 0.01),
    'colsample_bytree' : hp.quniform('colsample_bytree', 0.1, 1.0, 0.01)}

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trials)

print("Best: ", best)

100%|██████████| 50/50 [00:54<00:00,  1.09s/trial, best loss: 0.12318840579710155]
Best:  {'colsample_bytree': 0.5700000000000001, 'learning_rate': 0.16, 'max_depth': 5, 'min_child_weight': 7.0, 'n_estimators': 7, 'subsample': 0.46}


In [36]:
classifier = xgb.XGBClassifier(base_score=0.5,n_estimators = best['n_estimators'],
                            max_depth = best['max_depth'],
                            learning_rate = best['learning_rate'],
                            #gamma = best['gamma'],
                            min_child_weight = best['min_child_weight'],
                            subsample = best['subsample'],
                            colsample_bytree = best['colsample_bytree']
                            )

In [37]:
classifier.fit(df1 , df['Class'])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5700000000000001, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.16, max_delta_step=0, max_depth=5,
              min_child_weight=7.0, missing=nan, monotone_constraints='()',
              n_estimators=7, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=0.46,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [38]:
pred = classifier.predict(df1)

In [39]:
from sklearn.metrics import *

In [40]:
accuracy_score(pred , df['Class'])

0.8855072463768116

In [41]:
roc_auc_score(df['Class'] , classifier.predict_proba(df1)[:,1])

0.9492137335113666

In [21]:
# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

In [27]:
xgc1= xgb.XGBClassifier(base_score = 0.5 , learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)

In [28]:
%%time

param_comb = 5

from sklearn.model_selection import RandomizedSearchCV
#skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgc1, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=3, verbose=3, random_state=1001 )

# timing starts from this point for "start_time" variable
random_search.fit(df1, df['Class'])

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  15 out of  15 | elapsed:    3.3s finished


[22:25:13] WARNING: /workspace/src/learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


CPU times: user 469 ms, sys: 93.8 ms, total: 562 ms
Wall time: 3.79 s


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=XGBClassifier(base_score=0.5, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=0.02,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_...
                                           subsample=None, tree_method=None,
                                           validate_parameters=None,
                                  

In [29]:
random_search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1.0, gamma=5, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.02, max_delta_step=0, max_depth=3,
              min_child_weight=5, missing=nan, monotone_constraints='()',
              n_estimators=600, n_jobs=1, nthread=1, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, silent=True, subsample=1.0,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [30]:
pred_rcv = random_search.predict(df1)

In [32]:
accuracy_score(pred_rcv , df['Class'])

0.881159420289855

In [42]:
roc_auc_score(df['Class'] , random_search.predict_proba(df1)[:,1])

0.9585859960367745